### Simulate bulk competition experiments using empirical traits from Warringer 2003

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from bulk_simulation_code import run_pairwise_experiment, run_bulk_experiment
from bulk_simulation_code import CalcRelativeYield,CalcReferenceFrequency
from bulk_simulation_code import CalcTotalSelectionCoefficientLogit
from m3_model import CalcRelativeSaturationTime as CalcSaturationTimeExact
from m3_model import CalcFoldChangeWholePopulation

In [ ]:
### Update dependent parameters according to input
import os
import os.path
from os import path

## create export directory if necessary
## foldernames for output plots/lists produced in this notebook
import os
FIG_DIR = f'./figures/bulk_fitness/'
os.makedirs(FIG_DIR, exist_ok=True)
print("All  plots will be stored in: \n" + FIG_DIR)

In [ ]:
### execute script to load modules here
# I get some error with this command
# exec(open('setup_aesthetics.py').read()) 

# manual fix

FIGSIZE_A4 = (8.27, 11.69) # a4 format in inches

FIGWIDTH_TRIPLET = FIGSIZE_A4[0]*0.3*2
FIGHEIGHT_TRIPLET = FIGWIDTH_TRIPLET*0.75


In [ ]:
DATASET_COLOR = 'darkorange'


In [ ]:
SUFFIX_DATASET = 'warringer/'

FIG_DIR_DATASET = FIG_DIR + SUFFIX_DATASET
os.makedirs(FIG_DIR_DATASET, exist_ok=True)

OUTPUT_DIR_DATASET = './output/' + SUFFIX_DATASET
os.makedirs(OUTPUT_DIR_DATASET, exist_ok=True)

### Load wild-type traits

In [ ]:
INDEX_COL = [0,1,2,3,4]
list_na_representations = ['not_present', 'failed_to_compute']

In [ ]:
PCWS_TRAITS_WARRINGER = './output/df_M3_traits.csv'
df_warringer = pd.read_csv(PCWS_TRAITS_WARRINGER, header = 0, index_col= INDEX_COL,\
                                  float_precision=None, na_values=list_na_representations)


In [ ]:
### define default wild_type
df_wildtypes = df_warringer[df_warringer['is_wildtype']==True]

WILDTYPE = df_wildtypes.median(axis = 0, numeric_only = True)

### Load mutant data (averaged)

In [ ]:

PCWS_TRAITS_WARRINGER_AVERAGED = './output/df_M3_traits_averaged.csv'
df_averaged = pd.read_csv(PCWS_TRAITS_WARRINGER_AVERAGED, header = 0, float_precision=None)

In [ ]:
### assign wild-type label
def is_wildtype(row):
    genotype = row['genotype']
    
    if genotype == 'BY4741':
        return True
    else:
        return False
    

row = df_averaged.iloc[0]
is_wildtype(row)

In [ ]:
df_averaged['is_wildtype'] = df_averaged.apply(is_wildtype, axis =1)

In [ ]:
### append mutant values (averaged) to set of individual wild-type strains
df_knockouts = df_averaged[~df_averaged['is_wildtype']]
df_knockouts = df_knockouts
df_input = df_wildtypes.reset_index().append(df_knockouts.reset_index())

In [ ]:
### restore index
index_col_names = df_warringer.index.names
df_input = df_input.set_index(index_col_names)


### Load trait data into the standard form required by Michaels code

In [ ]:
n_knockouts = df_knockouts.shape[0]

In [ ]:
### growth rates
gs = np.zeros(n_knockouts+1)
gs[0] = WILDTYPE['gmax']
gs[1:] = df_knockouts['gmax'].values

### lag times
ls = np.zeros(n_knockouts+1)
ls[0] = WILDTYPE['lag']
ls[1:] = df_knockouts['lag'].values

### adjust units of time
gs = gs*60 # change units to growth rate per hour
ls = ls/60 # change units to hour

### yield
Ys = np.zeros(n_knockouts+1)
Ys[0] = WILDTYPE['yield']
#Ys[1:] = Ys[0] #switch off variation in yield
Ys[1:] = df_knockouts['yield'].values


### Define initial condition for bulk growth cycle

In [ ]:
### set initial resource concentrations

CONCENTRATION_GLUCOSE = 20/180 * 1e3 # concentrations are recored  in milliMolar, to match the units of yield
print(CONCENTRATION_GLUCOSE)

In [ ]:
### define default initial_OD
OD_START = 0.05  #df_warringer['od_start'].median()

### compare to initial OD in the monoculture cycles
fig, ax = plt.subplots(figsize = (FIGWIDTH_TRIPLET, FIGHEIGHT_TRIPLET))

ax = df_warringer['od_start'].hist(bins=41, color = DATASET_COLOR, alpha = 0.6, log = True, rasterized = True)


ax.axvline(OD_START, color = 'tab:red', label = f'median value: $N_0={OD_START:.3f}$')
ax.legend()
ax.set_xlabel('initial OD')
ax.set_ylabel('no. growth curves')

### Calculate effective yield

In [ ]:
from bulk_simulation_code import CalcRelativeYield

In [ ]:
### calculcate effective yields
nus = CalcRelativeYield(Ys, R0 = CONCENTRATION_GLUCOSE, N0 = OD_START)


### Simulate pairwise competition growth cycles (scenario A)

The frequencies for scenario A can be summarized as 

    frequency of the focal mutant strain: x2 = 1/N
        frequency of the wildtype strain: x1 = 1 - x2

where $N$ is some population size (number of cells or biomass?). Intutively, a spontanteous mutation initially forms only a small fraction $x0 = 1/N$ in the population. The values of the population size in nature are largely unknown, but can be approximated in two ways. 

- by the effective population size $N_e$, which is inferred from the genomic variation across a set of natural isolates, and leads to estimates of $N\approx 10^8$ cells [see papers by Howard Ochman]
- by the bottleneck size $N$ in laboratory evolution experiments like the LTEE, which leads to an estimate of $N=5\cdot 10^6$ cells. According to the first paper on the LTEE, there are $5\cdot10^5$ cells per ml at the starting point of the growth cycle, total volue is 10ml. 



In [ ]:
N = 1e6

In [ ]:
xs_pair, xs_pair_final, tsats_pair,fcs_both, _,_ = \
run_pairwise_experiment(gs=gs,ls=ls,nus = nus, g1=gs[0],l1=ls[0],nu1=nus[0],x0 = 1/N)

si_pair = CalcTotalSelectionCoefficientLogit(xs_pair,xs_pair_final)

### Simulate bulk competition with background mutants and added wild-type lineage  (scenario B2)

The frequencies for this scenario can be summarized as 

    frequency of the mutant straints:     xi = 1/(k+1)
    frequency of the wildtype strain:     x1 = 1/(k+1)

where `k` is the number of knockouts strain. Here all lineages, the mutants and the wild-type, have the same initial frequency. This roughly resembles scenario 'Bfull', but with a barcoded wild-type spiked into the culture.

In [ ]:
k = n_knockouts

In [ ]:
### set initial frequencies
xs = np.zeros_like(gs)
xs[1:] = 1/(k+1)           # mutant lineages
xs[0] = 1/(k+1)              # wildtype population



In [ ]:
print("Proportion of mutants: %.8f " % xs[1:].sum() )
print("Proportion of wild-type: %.8f " % xs[0] )

In [ ]:
## calculate final frequencies
xs, xs_final,tsat = run_bulk_experiment(gs=gs, ls = ls, nus =nus, xs=xs)

## compute total foldchange
fc_bulk = CalcFoldChangeWholePopulation(t=tsat,xs=xs,gs=gs,ls=ls)

## calculate total selection coefficient
si_bulk_B2 = CalcTotalSelectionCoefficientLogit(xs,xs_final)

## compute pairwise selection coefficient in bulk
xi1 = CalcReferenceFrequency(xs,ref_strains = [0]) 
xi1_final = CalcReferenceFrequency(xs_final,ref_strains = [0])
si1_bulk_B2 = CalcTotalSelectionCoefficientLogit(xi1,xi1_final)



### Calculate error to pairwise competition as ground truth

In [ ]:
%matplotlib inline

In [ ]:
### compare in a plot

fig, axes = plt.subplots(1,2, figsize = (2*FIGWIDTH_TRIPLET, FIGHEIGHT_TRIPLET), sharex = True, sharey=True)


ax = axes[0] # pairwise selection coefficient in bulk
x = si_pair 
y = si1_bulk_B2 - si_pair

ax.scatter(x[1:],y[1:], rasterized = True, color = 'dimgrey') 
ax.axhline(0, ls = '--', color = 'black')
ax.set_ylabel('s_21 bulk - s_21 pair')
ax.set_xlabel('s_21: pairwise experiment')

ax = axes[1] # total selection coefficient in bulk
x = si_pair
y = si_bulk_B2 - si_pair

ax.scatter(x[1:],y[1:], rasterized = True, color = 'dimgrey') 
ymin,ymax = ax.get_ylim()
yabs = np.max(np.abs([ymin,ymax]))
ax.set_ylim(-yabs,yabs)
ax.axhline(0, ls = '--', color = 'black')
ax.set_ylabel('s_2 bulk - s_21 pair')
ax.set_xlabel('s_21: pairwise experiment')

fig.tight_layout()

### Calculate error between total and pairwise selection coefficient

### Calculate trait components of the selection coefficient

In [ ]:
from m3_model import CalcApproxSijComponentsMultitype, CalcApproxSijComponents

In [ ]:
%%time

si1_bulk_growth = np.zeros_like(si1_bulk_B2)
si1_bulk_lag = np.zeros_like(si1_bulk_B2)
si1_bulk_coupling = np.zeros_like(si1_bulk_B2)

for i in range(len(gs)):
    si1_bulk_growth[i], si1_bulk_lag[i], si1_bulk_coupling[i] = CalcApproxSijComponentsMultitype(i,0,xs,gs,ls,nus)



In [ ]:
%%time

si_pair_growth = np.zeros_like(si_pair)
si_pair_lag = np.zeros_like(si_pair)
si_pair_coupling = np.zeros_like(si_pair)

for i in range(len(gs)):
        g1, l1, nu1 = gs[0], ls[0], nus[0]
        g2, l2, nu2 = gs[i], ls[i], nus[i] # get traits of the invader
        x0 = 1/N
        si_pair_growth[i], si_pair_lag[i], si_pair_coupling[i] =CalcApproxSijComponentsMultitype(1,0,
                                                xs=[1-x0,x0], gs = [g1,g2], ls= [l1,l2], nus = [nu1,nu2] )

### Break down the error by trait components

In [ ]:
### compare in a plot

fig, axes = plt.subplots(1,2, figsize = (2*FIGWIDTH_TRIPLET, FIGHEIGHT_TRIPLET), sharex = True, sharey = True)


ax = axes[0] # error from growth and lag component
x = si_pair
y = si1_bulk_growth + si1_bulk_lag - si_pair_growth - si_pair_lag

ax.scatter(x[1:],y[1:], rasterized = True, color = 'dimgrey')
ymin,ymax = ax.get_ylim()
yabs = np.max(np.abs([ymin,ymax]))
ax.set_ylim(-yabs,yabs)
ax.axhline(0, ls = '--', color = 'black')
ax.set_ylabel('delta s_growth + delta s_lag')
ax.set_xlabel('s_21: pairwise experiment')


ax = axes[1] # error from coupling component
x = si_pair
y = si1_bulk_coupling
ax.scatter(x[1:],y[1:], rasterized = True, color = 'dimgrey')

ymin,ymax = ax.get_ylim()
yabs = np.max(np.abs([ymin,ymax]))
ax.set_ylim(-yabs,yabs)
ax.axhline(0, ls = '--', color = 'black')
ax.set_ylabel('s_coupling: bulk experiment')
ax.set_xlabel('s_21: pairwise experiment')

fig.tight_layout()

In [ ]:
### compare size of the two error components
fig, axes = plt.subplots( figsize = (FIGWIDTH_TRIPLET, FIGHEIGHT_TRIPLET), sharex = True, sharey = True)


ax = axes # error from growth and lag component
x = si1_bulk_growth + si1_bulk_lag - si_pair_growth - si_pair_lag
y = si1_bulk_coupling

ax.scatter(x[1:],y[1:], rasterized = True, color = 'dimgrey')

## add axis labels
ax.set_xlabel('error growth and lag: delta s_growth + delta s_lag')
ax.set_ylabel('error from coupling: s_coupling')


### make square
xmin,xmax = ax.get_xlim()
ymin,ymax = ax.get_ylim()
xymin = np.min([xmin,ymin])
xymax = np.max([xmax,ymax])
xyabs = np.max(np.abs([xymin,xymax]))
ax.set_xlim(-xyabs,xyabs)
ax.set_ylim(-xyabs,xyabs)
## add diagonal line
ax.plot([-xyabs,xyabs],[-xyabs,xyabs], ls = '--', color = 'black')


fig.tight_layout()

### Plot the underlying changes in saturation time and fold-change

In [ ]:
### compute effective doubling times

taus = 1/gs
tau_bars_pair = np.zeros_like(taus)
x0 = 1/N

for i in range(len(gs)):
    tau1, tau2 = taus[0], taus[i]
    Y1, Y2 = Ys[0], Ys[i]
    x1, x2 = 1-x0, x0
    tau_bars_pair[i] = (x1/Y1 + x2/Y2)/(x1/Y1/tau1 + x2/Y2/tau2)

## for bulk experiment

tau_bar_bulk = np.sum(np.divide(xs,Ys)) / np.sum(np.divide(np.divide(xs,Ys),taus))

In [ ]:
### compare in a plot:  tau_bar vs log fold-change

fig, axes = plt.subplots(1,2, figsize = (2*FIGWIDTH_TRIPLET, FIGHEIGHT_TRIPLET), sharex = True, sharey = True)

ax = axes[0] # plot tau_bar data
x = si_pair
y = np.divide(tau_bar_bulk - tau_bars_pair, tau_bars_pair)

ax.scatter(x[1:],y[1:], rasterized = True, color = 'dimgrey')   # add data
ax.axhline(0, color = 'black', ls = '--')               # add horizontal line at zero
# center the plot vertically at zero
ymin,ymax = ax.get_ylim()
yabs = 1.1*np.max(np.abs([ymin,ymax]))
ax.set_ylim(-yabs,yabs)
# add axis labels
ax.set_ylabel('tau_bar: (bulk - pair)/pair')
ax.set_xlabel('s_21: pairwise competition')

ax = axes[1] # plot log foldchange data
x = si_pair
y = np.divide(np.log(fc_bulk) - np.log(fcs_pair), np.log(fcs_pair))

ax.scatter(x[1:],y[1:], rasterized = True, color = 'dimgrey')   # add data
ax.axhline(0, color = 'black', ls = '--')               # add horizontal line at zero
# center the plot vertically at zero
ymin,ymax = ax.get_ylim()
yabs = 1.1*np.max(np.abs([ymin,ymax]))
ax.set_ylim(-yabs,yabs)
# add axis labels
ax.set_ylabel('log foldchange: (bulk - pair)/pair')
ax.set_xlabel('s_21: pairwise competition')


fig.tight_layout()

In [ ]:
# show that error is mostly explained by tau_bar

### compare size of the two error components
fig, axes = plt.subplots(1,2, figsize = (2*FIGWIDTH_TRIPLET, FIGHEIGHT_TRIPLET), sharex=True)


ax = axes[0] # error from growth and lag component
x = np.divide(tau_bar_bulk - tau_bars_pair, tau_bars_pair)
y =  np.divide(si1_bulk_lag - si_pair_lag, si_pair_lag, where = si_pair_lag != 0)
ax.scatter(x[1:],y[1:], rasterized = True, color = 'dimgrey')
## add axis labels
ax.set_ylabel('error: delta s_lag')
ax.set_xlabel('tau_bar: (bulk - pair)/pair')


ax = axes[1]
x = np.divide(tau_bar_bulk - tau_bars_pair, tau_bars_pair)
y =  np.divide(si1_bulk_growth - si_pair_growth, si_pair_growth, where = si_pair_growth != 0)
ax.scatter(x[1:],y[1:], rasterized = True, color = 'dimgrey')


## add axis labels
ax.set_ylabel('error: delta s_growth ')
ax.set_xlabel('tau_bar: (bulk - pair)/pair')

for ax in axes:
    ax.ticklabel_format(useOffset=False, style = 'sci', scilimits = (0,0))


fig.tight_layout()

### Finding the optimal fraction of mutants in the bulk experiment

The frequencies for this scenario can be summarized as 

    frequency of the mutant strain s:     xi = x/k
    frequency of the wildtype strain:     x1 = 1-x
    
where `k` is the number of knockouts strain and `x` is the total proportion of mutant. This scenario is interpolation between the scenario **B1** with dominating wild-type (`x<<1`) and the scenario **Bfull** with no wildtype (`x=1`). 


In [ ]:
### choose range of frequencies to test
#xrange = np.geomspace(1/N,0.99, num = 30)
#xrange = np.linspace(0.01,0.99, num = 20)
xrange = np.linspace(0.01, k/(k-1),num = 20)

In [ ]:
%%time 

si_bulk_Bx = np.zeros((len(xrange),n_knockouts+1))
si1_bulk_Bx = np.zeros((len(xrange),n_knockouts+1))


for i in range(len(xrange)):
    x = xrange[i]
    
    ### set initial frequencies
    xs = np.zeros_like(gs)
    xs[1:] = x/k         # mutant lineages
    xs[0] = 1-x              # wildtype population

    ## calculate final frequencies
    xs, xs_final,_ = run_bulk_experiment(gs=gs, ls = ls, nus =nus, xs=xs)

    ## calculate total selection coefficient
    si_bulk_Bx[i,:] = CalcTotalSelectionCoefficientLogit(xs,xs_final)
    
    ## compute pairwise selection coefficient in bulk
    xi1 = CalcReferenceFrequency(xs,ref_strains = [0]) 
    xi1_final = CalcReferenceFrequency(xs_final,ref_strains = [0])
    si1_bulk_Bx[i,:] = CalcTotalSelectionCoefficientLogit(xi1,xi1_final)

### Plot error to pairwise competition as a function of mutant frequency

In [ ]:
truth = np.outer(np.ones(len(xrange)),si_pair) # need the right shape

## calculate error for pairwise selection coefficients: from higher order interactions
error_sij_abs = si1_bulk_Bx - truth 
error_sij_rel = np.divide(error_abs,truth, where = truth != 0)

## calculate error from total selection coefficients
error_si_abs = si_bulk_Bx - truth 
error_si_rel = np.divide(error_abs,truth, where = truth != 0)

In [ ]:
### use a line plot

fig, axes = plt.subplots(1,2, figsize = (2*FIGWIDTH_TRIPLET, FIGHEIGHT_TRIPLET), sharex = True, sharey = True)

ax = axes[0]
x = xrange
y = error_sij_abs
ax.plot(x,y[:,1:], rasterized = True, color = 'dimgrey')
ax.axhline(0, ls = '--', color = 'black')
ax.set_ylabel('s_21 bulk - s_21 pair')

ax = axes[1]
x = xrange
y = error_si_abs
ax.plot(x,y[:,1:], rasterized = True, color = 'dimgrey')
ax.axhline(0, ls = '--', color = 'black')
ax.set_ylabel('s_2 bulk - s_21 pair')

## center at zero
ymin,ymax = ax.get_ylim()
yabs = np.max(np.abs([ymin,ymax]))
ax.set_ylim(-yabs,yabs)


for ax in axes: 
    ax.set_xlabel('fraction of mutants in bulk competition: x')
    ax.axhline(0.01, color = 'red', ls = 'dotted')
    ax.axhline(-0.01, color = 'red', ls = 'dotted')
    
fig.tight_layout()

### Error II: How does error of frame of reference depend on mutant fraction?

In [ ]:
### use a line plot

fig, axes = plt.subplots(1,2, figsize = (2*FIGWIDTH_TRIPLET, FIGHEIGHT_TRIPLET),sharey= True)

ax = axes[0]
x = xrange
y = si_bulk_Bx - si1_bulk_Bx
ax.plot(x,y[:,1:], rasterized = True, color = 'dimgrey')
ax.axhline(0, ls = '--', color = 'black')
ax.set_ylabel('s_2 bulk - s_21 bulk')
ax.set_xlabel('fraction of mutants in bulk competition: x')


## center at zero
ymin,ymax = ax.get_ylim()
yabs = np.max(np.abs([ymin,ymax]))
ax.set_ylim(-yabs,yabs)

ax = axes[1]
x = si_pair 

### plot high frequency
ihigh,color_high = -1, 'dimgrey'
y = si_bulk_Bx[ihigh] - si1_bulk_Bx[ihigh]
ax.scatter(x[1:],y[1:], rasterized = True, color = color_high) 
### plot lower frequency
ilow,color_low = 4, 'red'
y = si_bulk_Bx[ilow] - si1_bulk_Bx[ilow]
ax.scatter(x[1:],y[1:], rasterized = True, color = color_low) 
## visualize on first axis
axes[0].axvline(xrange[ilow], color = color_low, ls = '--')
## add horizontal line
ax.axhline(0, ls = '--', color = 'black')
ax.set_ylabel('s_2 bulk - s_21 bulk')
ax.set_xlabel('s_21: pairwise experiment')

### center on zero
ymin,ymax = ax.get_ylim()
yabs = np.max(np.abs([ymin,ymax]))
ax.set_ylim(-yabs,yabs)

### add prediction
xmut = (1-xs[0])
error_predicted = -xmut*si_bulk_B2.mean()
print(error_predicted)
ax.axhline(error_predicted, color = 'red', ls = 'dotted')



fig.tight_layout()

### Error I: How do trait components  depend on mutant fraction?

In [ ]:
## calculate the change in tau_bar

tau_bar_range = np.zeros_like(xrange)

for i in range(len(xrange)):
    x = xrange[i] # read frequency
    
    ### set initial frequencies
    xs = np.zeros_like(gs)
    xs[1:] = x/k         # mutant lineages
    xs[0] = 1-x              # wildtype population
    

    tau_bar_range[i] = np.sum(np.divide(xs,Ys)) / np.sum(np.divide(np.divide(xs,Ys),taus))

In [ ]:
fig, ax = plt.subplots(figsize = (FIGWIDTH_TRIPLET,FIGHEIGHT_TRIPLET))

tau_wt = taus[0]
ax.plot(xrange,tau_bar_range, color = 'dimgrey')
ax.axhline(taus[0], color = 'darkorange', label = 'wildtype tau')
ax.set_xlabel('fraction of mutants: x')
ax.set_ylabel('mean doubling time: tau_bar')

ax.set_xlim(0,1)

In [ ]:
### calculate the change in the coupling component

n_subset = 300

s_lag=np.zeros((len(xrange), len(gs[:n_subset])))
s_growth=np.zeros((len(xrange), len(gs[:n_subset])))
s_coupling=np.zeros((len(xrange), len(gs[:n_subset])))

for i in range(len(xrange)):
    x = xrange[i] # read frequency
    
    ### set initial frequencies
    xs = np.zeros_like(gs)
    xs[1:] = x/k         # mutant lineages
    xs[0] = 1-x              # wildtype population
    
    for j in range(n_subset): #range(len(gs)):
        s_lag[i,j], s_growth[i,j], s_coupling[i,j] = CalcApproxSijComponentsMultitype(j,0,xs=xs,gs=gs,ls=ls,nus=nus)



In [ ]:
fig, axes = plt.subplots(1,2,figsize = (2*FIGWIDTH_TRIPLET,FIGHEIGHT_TRIPLET), sharey=True, sharex = True)

ax = axes[0]
y = s_lag + s_growth - si_pair_lag[:n_subset] - si_pair_growth[:n_subset]
ax.plot(xrange,y[:,1:], color = 'dimgrey', marker = 'x')
ax.set_xlabel('fraction of mutants: x')
ax.set_ylabel('delta s_growth + delta s_lag')

ax = axes[1]
y = s_coupling
ax.plot(xrange,y[:,1:], color = 'dimgrey', marker = 'x')
ax.set_xlabel('fraction of mutants: x')
ax.set_ylabel('s_21_coupling')

#ax.set_xscale('log')

### Visualize the frequency dependence as rotation and shrinking

In [ ]:
### compare in a plot

fig, axes = plt.subplots(1,2, figsize = (2*FIGWIDTH_TRIPLET, FIGHEIGHT_TRIPLET), sharex = True, sharey = True)


ax = axes[0] # error from growth and lag component
x = si_pair[:n_subset]

### plot at high frequency
ihigh, color_high = -1, 'dimgrey'
y = s_lag[ihigh] + s_growth[ihigh] - si_pair_lag[:n_subset] - si_pair_growth[:n_subset]
ax.scatter(x[1:],y[1:], rasterized = True, color = color_high)
### plot at lower frequency
ilow, color_low = 4, 'red'
y = s_lag[ilow] + s_growth[ilow] - si_pair_lag[:n_subset] - si_pair_growth[:n_subset]
ax.scatter(x[1:],y[1:], rasterized = True, color = color_low)

### add horizontal line
ymin,ymax = ax.get_ylim()
yabs = np.max(np.abs([ymin,ymax]))
ax.set_ylim(-yabs,yabs)
ax.axhline(0, ls = '--', color = 'black')
ax.set_ylabel('delta s_growth + delta s_lag')
ax.set_xlabel('s_21: pairwise experiment')


ax = axes[1] # error from coupling component
x = si_pair[:n_subset]

### plot at high frequency
y = s_coupling[ihigh]
ax.scatter(x[1:],y[1:], rasterized = True, color = color_high)
### plot at lower frequency
y = s_coupling[ilow]
ax.scatter(x[1:],y[1:], rasterized = True, color = color_low)

###
ymin,ymax = ax.get_ylim()
yabs = np.max(np.abs([ymin,ymax]))
ax.set_ylim(-yabs,yabs)
ax.axhline(0, ls = '--', color = 'black')
ax.set_ylabel('s_coupling: bulk experiment')
ax.set_xlabel('s_21: pairwise experiment')

fig.tight_layout()

### Calculate error bounds

In [ ]:
## calculate tau3
## calculate error bound x_min = (tau3-tau1)/tau3 * \theta/2
## calculate g3 